# GCaMP Centerline Regression
* Performs a regression with RFP and Centerline angle projections onto eig_basis

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.io as sio
%pylab inline

In [ ]:
# Read in worm data from SharedData .npz file
Worm = 5
worm_data = np.load('../SharedData/Worm_Angles/WormAngle'+str(Worm)+'.npz')

thetas = worm_data['proj_neural_thetas']
G = worm_data['G']
R = worm_data['R']
(neurons, time) = shape(G)
(time, eigen) = shape(thetas)

In [ ]:
for neuron in range(neurons):
    plt.plot(R[neuron,:])

In [ ]:
m = mean(R, axis=1)
v = np.var(R, axis=1)
print("mean: %1.5f, variance: %1.5f" % (np.mean(m), np.mean(v)))

In [ ]:
f ,ax = plt.subplots(1,2, figsize=(20,5*2))
ax[0].imshow(G, aspect=10,cmap="Paired")
ax[1].imshow(R, aspect=10,cmap="Paired")
for axis, title in zip(ax, [r'GCAMP', r'RFP']):
    axis.set_title(title)

In [ ]:
from sklearn import linear_model
#from scipy import stats
index_shuffle = np.arange(time)
np.random.shuffle(index_shuffle)

# take all but 100 for training
trainG = G#G[:,~index_shuffle[0:100]]
trainR = R#R[:,~index_shuffle[0:100]]
trainT = thetas #thetas[~index_shuffle[0:100],:]

print(shape(trainG))
# test
#testG = G[:,index_shuffle[0:100]]
#testR = R[:,index_shuffle[0:100]]
#testT = thetas[index_shuffle[0:100],:]
print(time)

vals = np.random.randint(0, len(G), 10)
f ,ax = plt.subplots(len(vals),1, figsize=(20,len(vals)*10))
for i, v in enumerate(vals):
    Y = trainG[v,:].reshape(time,1)
    X = np.concatenate((trainR[v,:].reshape(time,1),trainT),axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    #print('Coefficients: \n', regr.coef_)
    ax[i].scatter(trainR[v,:], Y, c= np.arange(len(X[:,1])), cmap='RdBu') # other possible colormaps are: YlOrBr, YlGnBu, 
    ax[i].plot(trainR[v,:],Y, lw=0.2)
    ax[i].plot(trainR[v,:], regr.predict(X))
    ax[i].set_title('Neuron: ' + str(v) + ', R_val: ' + "%.3f" % regr.score(X,Y))
